In [8]:
import json
import time

import requests
from rdflib import Graph, URIRef, Literal, Namespace
from tqdm import tqdm
from rdflib.namespace import RDF, SKOS,RDFS

In [16]:
g = Graph()
#space = "http://projet.fr/perso/"
api = "http://localhost/html/preprocessId.php?name="

errors = []
#g.bind("", "http://projet.fr/perso#")

#ns_perso_data = Namespace(space)
#ns_perso_schema = Namespace('"http://projet.fr/perso/"')
"""g.bind("skos", SKOS)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)"""

#ns_film_data = Namespace('http://projet.fr/films_data/')
#ns_film_schema = Namespace('http://projet.fr/films_schema/')
ns_perso_schema = Namespace('http://projet.fr/perso_schema/')
ns_perso_data = Namespace('http://projet.fr/perso_data/')

g.bind('p_schema', ns_perso_schema)
g.bind("", ns_perso_data)


def getHero(k):
    url = api + str(k)
    reponse = requests.get(url)
    content = reponse.content
    y = json.loads(content)[0]
    return y

def getRace(y):
    race = y["appearance"]["race"]
    if race!="" and race!="-":
        raceNode = ns_perso_schema[''.join(letter for letter in race if letter.isalnum())]
        g.add((raceNode, SKOS.prefLabel, Literal(race)))
        g.add((raceNode, RDF.type, SKOS.Concept))

def getRelatives(y):
    relatives = y["connections"]["relatives"]
    for relative in relatives:
        relativeNode = ns_perso_data['P' + ''.join(letter for letter in relative.lower() if letter.isalnum())]
        g.add((relativeNode, ns_perso_schema.name, Literal(relative)))
        g.add((relativeNode, RDF.type, ns_perso_schema["Character"]))


def getGroups(y):
    groups = y["connections"]["group-affiliation"]
    for group in groups:
        groupNode = ns_perso_data[''.join(letter for letter in group if letter.isalnum())]
        g.add((groupNode, RDF.type, OWL.Class))
        g.add((groupNode, RDFS.label, Literal(group)))
        #g.add((groupNode, RDFS.subClassOf, URIRef(space + "Group")))
        #g.add((groupNode, RDFS.subClassOf, ns_perso_schema['Group']))


def getHeroNode(y):
    superName = y["name"]
    name = y["biography"]["full-name"]
    id = name.lower() + superName.lower()
    heroNode = ns_perso_data['H'+''.join(letter for letter in id  if letter.isalnum())]
    conceptNode = ns_perso_data['C'+''.join(letter for letter in superName if letter.isalnum())]
    persoNode = ns_perso_data['P'+''.join(letter for letter in id if letter.isalnum())]
    g.add((heroNode, ns_perso_schema.name, Literal(superName)))
    g.add((conceptNode, RDFS.subClassOf, ns_perso_schema["Character"]))
    g.add((conceptNode, RDF.type, SKOS.Concept))
    g.add((conceptNode, SKOS.prefLabel, Literal(superName)))
    if name != "":
        g.add((persoNode, ns_perso_schema.name, Literal(name)))
    g.add((persoNode, RDF.type, ns_perso_schema["Character"]))
    g.add((persoNode, RDF.type, ns_perso_schema["Fictional"]))
    g.add((heroNode, RDF.type, ns_perso_schema["Character"]))
    g.add((heroNode, RDF.type, ns_perso_schema["Fictional"]))


def getPublisher(y):
    publisher = y["biography"]["publisher"]
    publisherNode = ns_perso_data[''.join(letter for letter in publisher if letter.isalnum())]
    g.add((publisherNode, RDF.type, SKOS.Concept))
    g.add((publisherNode, RDFS.subClassOf, ns_perso_schema['Publisher']))
    g.add((publisherNode, SKOS.prefLabel, Literal(publisher)))


def getColor(y):
    colorsEye = y["appearance"]["eye-color"]
    colorsHair = y["appearance"]["hair-color"]
    for colorEye in colorsEye:
        eyeNode = ns_perso_data[''.join(letter for letter in colorEye if letter.isalnum())]
        g.add((eyeNode, SKOS.prefLabel, Literal(colorEye)))
        g.add((eyeNode, RDFS.subClassOf, ns_perso_schema['Color']))
        g.add((eyeNode, RDF.type, SKOS.Concept))
    for colorHair in colorsHair:
        hairNode = ns_perso_data[''.join(letter for letter in colorHair if letter.isalnum())]
        g.add((hairNode, RDF.type, SKOS.Concept))
        g.add((hairNode, SKOS.prefLabel, Literal(colorHair)))
        g.add((hairNode, RDFS.subClassOf, ns_perso_schema['Color']))


def getOccupation(y):
    occupations = y["work"]["occupation"]
    for occupation in occupations:
        occupationNode = ns_perso_data[''.join(letter for letter in occupation if letter.isalnum())]
        g.add((occupationNode, RDF.type, SKOS.Concept))
        g.add((occupationNode, SKOS.prefLabel, Literal(occupation)))
        g.add((occupationNode, RDFS.subClassOf, ns_perso_schema['Occupation']))


def createNodes(k):
    try:   
        y = getHero(k)
    except Exception as e:
        print(e)
        errors.append(e)
    else:
        getHeroNode(y)
        getPublisher(y)
        getOccupation(y)
        getColor(y)
        getRelatives(y)




In [17]:
for k in tqdm(range(1, 732)):
    createNodes(k)
    time.sleep(0.1)

g.serialize(destination="database.ttl", format="turtle")

100%|██████████| 731/731 [13:24<00:00,  1.10s/it]


<Graph identifier=N2219df464bf54c2d8e18f9ccf6245137 (<class 'rdflib.graph.Graph'>)>

In [10]:
errors

[]

In [54]:
from rdflib.plugins.sparql import prepareQuery

query = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://projet.fr/perso#>
SELECT ?x ?y ?z where {
    ?x a :Character.
    ?x ?y ?z.
    
    
}
'''
queryGroup = prepareQuery(query)
# Create a new RDF graph to store the query result
result_graph = Graph()
for row in g.query(queryGroup):
    result_graph.add(row)
res = result_graph.serialize()
print(res)

@prefix ns1: <http://projet.fr/perso#> .

<http://projet.fr/perso_data/H1> a ns1:Character ;
    ns1:name "A-Bomb" .

<http://projet.fr/perso_data/H10> a ns1:Character ;
    ns1:name "Agent Bob" .

<http://projet.fr/perso_data/H100> a ns1:Character ;
    ns1:name "Black Flash" .

<http://projet.fr/perso_data/H101> a ns1:Character ;
    ns1:name "Black Goliath" .

<http://projet.fr/perso_data/H102> a ns1:Character ;
    ns1:name "Black Knight III" .

<http://projet.fr/perso_data/H103> a ns1:Character ;
    ns1:name "Black Lightning" .

<http://projet.fr/perso_data/H104> a ns1:Character ;
    ns1:name "Black Mamba" .

<http://projet.fr/perso_data/H105> a ns1:Character ;
    ns1:name "Black Manta" .

<http://projet.fr/perso_data/H106> a ns1:Character ;
    ns1:name "Black Panther" .

<http://projet.fr/perso_data/H107> a ns1:Character ;
    ns1:name "Black Widow" .

<http://projet.fr/perso_data/H108> a ns1:Character ;
    ns1:name "Black Widow II" .

<http://projet.fr/perso_data/H109> a ns

In [48]:
from rdflib.plugins.sparql import prepareQuery

query = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://projet.fr/perso#>
SELECT (count(?x) as ?total) where {
    ?x a :Character.
}
'''
queryGroup = prepareQuery(query)
# Create a new RDF graph to store the query result
result_graph = Graph()
g.query(queryGroup).serialize()


b'<?xml version="1.0" encoding="utf-8"?>\n<sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xml="http://www.w3.org/XML/1998/namespace"><head><variable name="total"></variable></head><results><result><binding name="total"><literal datatype="http://www.w3.org/2001/XMLSchema#integer">1458</literal></binding></result></results></sparql>'

In [49]:
errors

[269, 324]

In [50]:
file_object = open("persos.ttl", "w")
file_object.write(res)
file_object.close()

In [80]:
print(g.serialize())

@prefix : <http://projet.fr/perso#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

: a owl:Class,
        skos:Concept ;
    rdfs:label "-" ;
    rdfs:subClassOf :Alignment,
        :Color,
        :Group,
        :Publisher ;
    skos:prefLabel "",
        "-" .

:198 a owl:Class ;
    rdfs:label "198" ;
    rdfs:subClassOf :Group .

:abcstudios a skos:Concept ;
    rdfs:subClassOf :Publisher ;
    skos:prefLabel "ABC Studios" .

:abesapien a owl:Class ;
    rdfs:label "Abe Sapien" ;
    rdfs:subClassOf :Group .

:absorbingman a owl:Class ;
    rdfs:label "Absorbing Man" ;
    rdfs:subClassOf :Group .

:acolytes a owl:Class ;
    rdfs:label "Acolytes" ;
    rdfs:subClassOf :Group .

:acolytesattimeofdeath a owl:Class ;
    rdfs:label "Acolytes at time of death" ;
    rdfs:subClassOf :Group .

:activememberofthexmen a owl:Class ;
    rdfs:label "Active member of the X-Men"

In [41]:
createNodes(1)
print(g.serialize())

@prefix : <http://projet.fr/perso#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

: a owl:Class ;
    rdfs:label "['Hulk Family; Excelsior', 'Avengers ; formerly partner of the Hulk', 'Captain America and Captain Marvel; Teen Brigade; ally of Rom']" .

:MarvelComics a skos:Concept ;
    skos:prefLabel "Marvel Comics" .

:NoHair a skos:Concept ;
    skos:prefLabel "No Hair" .

:Yellow a skos:Concept ;
    skos:prefLabel "Yellow" .

:good a skos:Concept ;
    skos:prefLabel "good" .

<http://projet.fr/perso_data/1> a skos:Concept ;
    skos:prefLabel "A-Bomb" .

<http://projet.fr/perso_data/H1> :name "A-Bomb" .

<http://projet.fr/perso_data/P1> :name "Richard Milhouse Jones" .


In [26]:

heroNode = URIRef("http://projet.fr/perso_data/H" + id)
conceptNode = URIRef("http://projet.fr/perso_data/" + id)
persoNode = URIRef("http://projet.fr/perso_data/P" + id)
publisherNode = URIRef(space + ''.join(letter for letter in publisher if letter.isalnum()))
alignmentNode = URIRef(space + ''.join(letter for letter in alignment if letter.isalnum()))
occupationNode = URIRef(space + ''.join(letter for letter in occupation if letter.isalnum()))
groupNode = URIRef(space + ''.join(letter for letter in group if letter.isalnum()))
eyeNode = URIRef(space + ''.join(letter for letter in colorEye if letter.isalnum()))
hairNode = URIRef(space + ''.join(letter for letter in colorHair if letter.isalnum()))

g.add((heroNode, URIRef("http://projet.fr/perso#name"), Literal(superName)))

g.add((conceptNode, RDF.type, SKOS.Concept))
g.add((conceptNode, SKOS.prefLabel, Literal(superName)))

g.add((persoNode, URIRef("http://projet.fr/perso#name"), Literal(name)))

g.add((publisherNode, RDF.type, SKOS.Concept))
g.add((publisherNode, SKOS.prefLabel, Literal(publisher)))

g.add((alignmentNode, RDF.type, SKOS.Concept))
g.add((alignmentNode, SKOS.prefLabel, Literal(alignment)))

g.add((occupationNode, RDF.type, SKOS.Concept))
g.add((occupationNode, SKOS.prefLabel, Literal(occupation)))

g.add((groupNode, RDF.type, OWL.Class))
g.add((groupNode, RDFS.label, Literal(group)))

g.add((hairNode, RDF.type, SKOS.Concept))
g.add((hairNode, SKOS.prefLabel, Literal(colorHair)))

g.add((eyeNode, RDF.type, SKOS.Concept))
g.add((eyeNode, SKOS.prefLabel, Literal(colorEye)))

<Graph identifier=N11bc81b36dfe42b880aaf8c5d796bee9 (<class 'rdflib.graph.Graph'>)>

In [27]:
print(g.serialize())

@prefix : <http://projet.fr/perso#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

: a owl:Class,
        skos:Concept ;
    rdfs:label "['Batman Family', 'Justice League Unlimited']" ;
    skos:prefLabel "['-']" .

:Black a skos:Concept ;
    skos:prefLabel "Black" .

:Blue a skos:Concept ;
    skos:prefLabel "Blue" .

:DCComics a skos:Concept ;
    skos:prefLabel "DC Comics" .

:good a skos:Concept ;
    skos:prefLabel "good" .

<http://projet.fr/perso_data/69> a skos:Concept ;
    skos:prefLabel "Batman" .

<http://projet.fr/perso_data/H69> :name "Batman" .

<http://projet.fr/perso_data/P69> :name "Terry McGinnis" .


In [ ]:

api = "http://superheroapi.com/api/2521867701306092/"
g = Graph()
g.bind(":", "http://projet.fr/perso#")
g.bind("skos", SKOS)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)

for k in range(1, 732):
    print(k)
    url = api + str(k)
    reponse = requests.get(url)
    content = reponse.content
    y = json.loads(content)
    gender = y["appearance"]["gender"]
    race = y["appearance"]["race"]
    raceName = race.replace(" ", "")
    genderName = gender.replace(" ", "")
    newRace = URIRef("http://projet.fr/perso#" + raceName)
    newGender = URIRef("http://projet.fr/perso#" + genderName)
    name = Literal(race)
    gend = Literal(gender)
    g.add((newRace, RDF.type, SKOS.Concept))
    g.add((newRace, SKOS.prefLabel, name))
    g.add((newGender, RDF.type, OWL.Class))
    g.add((newGender, RDFS.label, gend))
    time.sleep(1)

print(g.serialize())

In [ ]:
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, SKOS, OWL, RDFS
import time

api = "http://superheroapi.com/api/2521867701306092/"
g = Graph()
g.bind(":", "http://projet.fr/perso#")
g.bind("skos", SKOS)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)

for k in range(1, 732):
    print(k)
    url = api + str(k)
    reponse = requests.get(url)
    content = reponse.content
    y = json.loads(content)
    gender = y["appearance"]["gender"]
    race = y["appearance"]["race"]
    raceName = race.replace(" ", "")
    genderName = gender.replace(" ", "")
    newRace = URIRef("http://projet.fr/perso#" + raceName)
    newGender = URIRef("http://projet.fr/perso#" + genderName)
    name = Literal(race)
    gend = Literal(gender)
    g.add((newRace, RDF.type, SKOS.Concept))
    g.add((newRace, SKOS.prefLabel, name))
    g.add((newGender, RDF.type, OWL.Class))
    g.add((newGender, RDFS.label, gend))
    time.sleep(1)

print(g.serialize())

In [20]:
url = "http://superheroapi.com/api/2521867701306092/732"
reponse = requests.get(url)
content = reponse.content
y = json.loads(content)
y

{'response': 'success',
 'id': '732',
 'name': 'Ironman',
 'powerstats': {'intelligence': '100',
  'strength': '85',
  'speed': '58',
  'durability': '85',
  'power': '100',
  'combat': '64'},
 'biography': {'full-name': 'Tony Stark',
  'alter-egos': 'No alter egos found.',
  'aliases': ['Iron Knight',
   'Hogan Potts',
   'Spare Parts Man',
   'Cobalt Man II',
   'Crimson Dynamo',
   'Ironman'],
  'place-of-birth': 'Long Island, New York',
  'first-appearance': 'Tales of Suspence #39 (March, 1963)',
  'publisher': 'Marvel Comics',
  'alignment': 'good'},
 'appearance': {'gender': 'Male',
  'race': 'Human',
  'height': ["6'6", '198 cm'],
  'weight': ['425 lb', '191 kg'],
  'eye-color': 'Blue',
  'hair-color': 'Black'},
 'work': {'occupation': 'Inventor, Industrialist; former United States Secretary of Defense',
  'base': 'Seattle, Washington'},
 'connections': {'group-affiliation': 'Avengers, Illuminati, Stark Resilient; formerly S.H.I.E.L.D., leader of Stark Enterprises, the Pro-Regis

In [25]:
url = "http://superheroapi.com/api/2521867701306092/"
url += str(732)
url


'http://superheroapi.com/api/2521867701306092/732'

In [22]:
y["appearance"]["race"]

'Human'

In [37]:
concepts = g.serialize()
concepts = concepts.replace("::", ":")
print(concepts)

@prefix : <http://projet.fr/perso#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

<http://projet.fr/perso#-> a owl:Class ;
    rdfs:label "-" .

:Alien a skos:Concept ;
    skos:prefLabel "Alien" .

:Alpha a skos:Concept ;
    skos:prefLabel "Alpha" .

:Amazon a skos:Concept ;
    skos:prefLabel "Amazon" .

:Android a skos:Concept ;
    skos:prefLabel "Android" .

:Animal a skos:Concept ;
    skos:prefLabel "Animal" .

:Asgardian a skos:Concept ;
    skos:prefLabel "Asgardian" .

:Atlantean a skos:Concept ;
    skos:prefLabel "Atlantean" .

:Bizarro a skos:Concept ;
    skos:prefLabel "Bizarro" .

:BlackRacer a skos:Concept ;
    skos:prefLabel "Black Racer" .

:Bolovaxian a skos:Concept ;
    skos:prefLabel "Bolovaxian" .

:Clone a skos:Concept ;
    skos:prefLabel "Clone" .

:CosmicEntity a skos:Concept ;
    skos:prefLabel "Cosmic Entity" .

:Cyborg a skos:Concept ;
  

In [38]:
file_object = open("concepts.ttl", "w")
file_object.write(concepts)
file_object.close()

In [40]:
g = Graph()
g.parse("concepts.ttl", format="turtle")


<Graph identifier=N01034edb0c7c4400a0ea115d87552931 (<class 'rdflib.graph.Graph'>)>

In [41]:
len(g)

126

In [42]:
print(g.serialize())

@prefix : <http://projet.fr/perso#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

:Alien a skos:Concept ;
    skos:prefLabel "Alien" .

:Alpha a skos:Concept ;
    skos:prefLabel "Alpha" .

:Amazon a skos:Concept ;
    skos:prefLabel "Amazon" .

:Android a skos:Concept ;
    skos:prefLabel "Android" .

:Animal a skos:Concept ;
    skos:prefLabel "Animal" .

:Asgardian a skos:Concept ;
    skos:prefLabel "Asgardian" .

:Atlantean a skos:Concept ;
    skos:prefLabel "Atlantean" .

:Bizarro a skos:Concept ;
    skos:prefLabel "Bizarro" .

:BlackRacer a skos:Concept ;
    skos:prefLabel "Black Racer" .

:Bolovaxian a skos:Concept ;
    skos:prefLabel "Bolovaxian" .

:Clone a skos:Concept ;
    skos:prefLabel "Clone" .

:CosmicEntity a skos:Concept ;
    skos:prefLabel "Cosmic Entity" .

:Cyborg a skos:Concept ;
    skos:prefLabel "Cyborg" .

:Czarnian a skos:Concept ;
    sko

In [43]:
distinct_subjects = set(g.subjects())
len(distinct_subjects)

63

In [45]:
URI = URIRef("http://projet.fr/perso#Race")
for subject in distinct_subjects:
    # Create a new triple: subject is a subclass of itself
    g.add((subject, RDFS.subClassOf, URI))
print(g.serialize())

@prefix : <http://projet.fr/perso#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

:Alien a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Alien" .

:Alpha a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Alpha" .

:Amazon a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Amazon" .

:Android a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Android" .

:Animal a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Animal" .

:Asgardian a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Asgardian" .

:Atlantean a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Atlantean" .

:Bizarro a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Bizarro" .

:BlackRacer a skos:Concept ;
    rdfs:subClassOf :Race ;
    skos:prefLabel "Black Racer" .

:Bolovaxian a skos:Concep

In [47]:
file_object = open("concepts.ttl", "w")
file_object.write(g.serialize())
file_object.close()